# 🚀 Projeto: Construção da Tabela Gold - Dimensão Produto

## 📌 Visão Geral

Este módulo representa a construção da **tabela de dimensão dim_produto** na camada **Gold** do projeto de Data Lakehouse. A tabela é derivada da Silver e foi desenhada para armazenar as informações relevantes dos produtos vendidos, com **altos padrões de qualidade de dados**, **otimização de leitura** e **governança**.

---

## 🎯 Objetivo

Criar uma tabela dimensional que represente os produtos, garantindo:

- 📦 Dados únicos e limpos de cada produto (ItemID)
- ⚙️ Estrutura otimizada para consultas analíticas
- 📈 Rastreabilidade com controle de criação (Created_at)
- 🔄 Atualizações eficientes via **MERGE Delta**
- 🚀 Alto desempenho com **Z-ORDER** e **reparticionamento estratégico**

---

## 📐 Detalhes Técnicos

- **Fonte**: Camada Silver (tabela_prata_desnormalizadas)
- **Destino**: Camada Gold (gold.dim_produto)
- **Particionamento**: ItemID (100 partições)
- **Chave de Negócio**: ItemID
- **Colunas da Dimensão Produto**:
  - ItemID (identificador do produto)
  - ProductName (nome do produto)
  - Created_at (timestamp de inserção na tabela Gold)

---

## ⚙️ Etapas do Pipeline

1. **Leitura da Tabela Silver** com controle de partições.
2. **Seleção das Colunas Relevantes**: apenas ItemID e ProductName.
3. **Limpeza de Dados**: remoção de duplicatas e valores nulos.
4. **Adição de Auditoria Temporal** com a coluna Created_at.
5. **Cache dos Dados** para melhorar performance nas operações subsequentes.
6. **Auditoria Inicial**: contagem dos produtos únicos existentes.
7. **Aplicação do MERGE Delta**: inserção e atualização baseada em ItemID.
8. **Auditoria Final**: nova contagem dos produtos únicos pós-processamento.
9. **Otimização com Z-ORDER** por ItemID.
10. **Limpeza com VACUUM** para liberação de espaço.
11. **Registro no Catálogo Hive/Unity Catalog** para consultas SQL.

---

## ✅ Benefícios Técnicos Aplicados

| Técnica                      | Finalidade                                                                |
|----------------------------  | --------------------------------------------------------------------------|
| MERGE Delta                | Evita duplicidade e garante atualizações incrementais                     |
| Reparticionamento            | Aumenta performance de escrita/leitura por chave de negócio (ItemID)    |
| dropDuplicates + na.drop | Garante consistência e integridade dos dados                              |
| Z-ORDER                    | Melhora tempo de resposta nas consultas filtradas                         |
| VACUUM                     | Reduz uso de armazenamento eliminando arquivos obsoletos                  |
| Registro no catálogo         | Permite uso via SQL e dashboards com governança centralizada              |

---

## 🧱 Modelo Estrela (Star Schema)

Esta tabela é utilizada como uma **Dimensão** em um **Esquema Estrela**, onde será relacionada à Tabela Fato de Vendas através da chave ItemID, permitindo análises como:

- Total de vendas por produto
- Produtos mais vendidos por período
- Comparações de performance entre categorias

---


In [0]:
from pyspark.sql.functions import col, current_timestamp
from delta.tables import DeltaTable


# 1. Definição dos caminhos das tabelas Delta na camada Silver e Gold
# ------------------------------------------
# Estes caminhos apontam para as localizações físicas no Data Lake onde os dados Delta são armazenados.
# São usados para leitura, escrita e operações de manutenção da tabela de dimensão produto.
SILVER_PATH = "abfss://silver@dlsprojetofixo.dfs.core.windows.net/tabela_prata_desnormalizadas"
GOLD_DIM_PRODUCT_PATH = "abfss://gold@dlsprojetofixo.dfs.core.windows.net/gold_dim_produto"
GOLD_DIM_PRODUCT_TABLE = "gold.dim_produto"

# ------------------------------------------
# 2. Verifica se o banco de dados 'gold' existe e cria caso não exista
# ------------------------------------------
# Criar o banco de dados 'gold' se não existir para garantir que a tabela seja registrada corretamente.
spark.sql("CREATE DATABASE IF NOT EXISTS gold")

# ------------------------------------------
# 3. Leitura dos dados da camada Silver (prata) no formato Delta
# ------------------------------------------
# Carrega os dados desnormalizados da camada intermediária Silver, que já passaram por tratamento inicial.
silver_df = spark.read.format("delta").load(SILVER_PATH)

# ------------------------------------------
# 4. Reparticionamento dos dados por 'ItemID' para balanceamento
# ------------------------------------------
# Reparticiona os dados para melhorar performance de escrita e leitura, especialmente em joins ou filtros por 'ItemID'.
silver_df = silver_df.repartition(100, "ItemID")  # Ajustar a quantidade conforme o volume de dados

# ------------------------------------------
# 5. Seleção das colunas relevantes para a dimensão Produto
# ------------------------------------------
# Aqui extraímos apenas os campos necessários para a construção da dimensão: identificador e nome do produto.
dim_product_df = silver_df.select("ItemID", "ProductName")

# ------------------------------------------
# 6. Remoção de duplicatas e valores nulos
# ------------------------------------------
# Elimina duplicidades por 'ItemID' e descarta linhas com nomes de produtos nulos.
dim_product_df = dim_product_df.dropDuplicates(["ItemID"])
dim_product_df = dim_product_df.na.drop(subset=["ProductName"])

# ------------------------------------------
# 7. Inclusão de coluna de auditoria 'Created_at'
# ------------------------------------------
# A coluna 'Created_at' registra a data/hora da carga, útil para rastreabilidade dos dados.
dim_product_df = dim_product_df.withColumn("Created_at", current_timestamp())

# ------------------------------------------
# 8. Cache do DataFrame para otimizar performance em múltiplas ações subsequentes
# ------------------------------------------
# Evita recomputações desnecessárias durante a execução do pipeline.
dim_product_df.cache()

# ------------------------------------------
# 9. Auditoria: contagem de registros distintos antes do MERGE
# ------------------------------------------
# Carrega a versão atual da dimensão, caso exista, e conta o número de produtos distintos.
dim_product_count_before = 0
if DeltaTable.isDeltaTable(spark, GOLD_DIM_PRODUCT_PATH):
    existing_dim_product_df = spark.read.format("delta").load(GOLD_DIM_PRODUCT_PATH)
    dim_product_count_before = existing_dim_product_df.select("ItemID").distinct().count()

print(f"Total de registros distintos antes do processamento (Gold Dim Produto): {dim_product_count_before}")

# ------------------------------------------
# 10. Habilita a atualização automática de schema no Delta Lake
# ------------------------------------------
# Permite que o Delta aceite novos campos durante a escrita sem erro de schema incompatível.
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

# ------------------------------------------
# 11. MERGE ou criação da tabela Gold Dim Produto
# ------------------------------------------
# Se a tabela já existir, faz um MERGE para inserir/atualizar registros com base no 'ItemID'.
# Caso contrário, cria a tabela do zero com os dados atuais.
if DeltaTable.isDeltaTable(spark, GOLD_DIM_PRODUCT_PATH):
    dim_product_table = DeltaTable.forPath(spark, GOLD_DIM_PRODUCT_PATH)

    dim_product_table.alias("target").merge(
        dim_product_df.alias("source"), "target.ItemID = source.ItemID"
    ).whenMatchedUpdate(set={
        "ProductName": col("source.ProductName"),
        "Created_at": col("source.Created_at")
    }).whenNotMatchedInsert(values={
        "ItemID": col("source.ItemID"),
        "ProductName": col("source.ProductName"),
        "Created_at": col("source.Created_at")
    }).execute()

    print("Dados adicionados com sucesso na tabela Gold Dim Produto.")
else:
    dim_product_df.write.format("delta").mode("append").option("mergeSchema", "true").save(GOLD_DIM_PRODUCT_PATH)
    print("Tabela Gold Dim Produto criada com sucesso.")

# ------------------------------------------
# 12. Auditoria: contagem de registros distintos após o MERGE
# ------------------------------------------
# Compara o total de registros distintos depois da carga para garantir a integridade e rastreabilidade.
dim_product_count_after = spark.read.format("delta").load(GOLD_DIM_PRODUCT_PATH).select("ItemID").distinct().count()
print(f"Total de registros distintos após o processamento (Gold Dim Produto): {dim_product_count_after}")

# ------------------------------------------
# 13. Otimização com Z-ORDER para consultas eficientes
# ------------------------------------------
# O Z-ORDER organiza fisicamente os dados para melhorar a performance de leitura por colunas consultadas frequentemente.
spark.sql(f"OPTIMIZE delta.`{GOLD_DIM_PRODUCT_PATH}` ZORDER BY (ItemID)")
print("Tabela Gold Dim Produto otimizada com Z-ORDER para melhorar a performance.")

# ------------------------------------------
# 14. VACUUM para remoção de arquivos obsoletos
# ------------------------------------------
# Executa a limpeza de arquivos antigos não referenciados pela tabela nos últimos 7 dias (168 horas).
# Libera espaço e mantém a eficiência do armazenamento.
spark.sql(f"VACUUM delta.`{GOLD_DIM_PRODUCT_PATH}` RETAIN 168 HOURS")
print("VACUUM executado para liberar espaço de armazenamento.")

# ------------------------------------------
# 15. Registro da tabela no catálogo (Metastore) do Spark
# ------------------------------------------
# Permite que a tabela seja consultada via SQL, conectores de BI e outros sistemas que usam o catálogo.
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {GOLD_DIM_PRODUCT_TABLE}
USING DELTA
LOCATION '{GOLD_DIM_PRODUCT_PATH}'
""")
print("Tabela Gold Dim Produto registrada no catálogo com sucesso.")

# ------------------------------------------
# 16. Resumo final das contagens para auditoria
# ------------------------------------------
# Apresenta de forma clara o antes e depois da quantidade de produtos únicos processados.
print("\nResumo das contagens distintas:")
print(f"Total distinto antes do processamento (Gold Dim Produto): {dim_product_count_before}")
print(f"Total distinto após o processamento (Gold Dim Produto): {dim_product_count_after}")


Total de registros distintos antes do processamento (Gold Dim Produto): 29945
Dados adicionados com sucesso na tabela Gold Dim Produto.
Total de registros distintos após o processamento (Gold Dim Produto): 29945
Tabela Gold Dim Produto otimizada com Z-ORDER para melhorar a performance.
VACUUM executado para liberar espaço de armazenamento.
Tabela Gold Dim Produto registrada no catálogo com sucesso.

Resumo das contagens distintas:
Total distinto antes do processamento (Gold Dim Produto): 29945
Total distinto após o processamento (Gold Dim Produto): 29945


In [0]:
%sql
SELECT * FROM gold.dim_produto LIMIT 20

ItemID,ProductName,Created_at
0191a621-1663-465a-826c-872de1b756f1,Headphones,2025-04-22T02:36:09.232Z
01d9c3e2-6f2f-433e-8df0-90c3f3cb0851,Smartwatch,2025-04-22T02:36:09.232Z
020a1034-09ca-473b-a0f7-db3804659057,Tablet,2025-04-22T02:36:09.232Z
020c3d9a-5162-463d-a65f-7b3d61b88df9,External Hard Drive,2025-04-22T02:36:09.232Z
026b32c6-fbac-4a22-85de-ee4e6ac3fc8c,USB Cable,2025-04-22T02:36:09.232Z
02ed91fb-6890-4840-873d-ff6f7d31795a,Power Supply,2025-04-22T02:36:09.232Z
034d27d4-857e-4a0e-83b1-609a7e06c788,Scanner,2025-04-22T02:36:09.232Z
042f561d-78b4-4ab9-a40e-2e9bb25e8173,USB Cable,2025-04-22T02:36:09.232Z
04dff3d1-4b83-4631-897a-0ad73005107d,Webcam,2025-04-22T02:36:09.232Z
067395c3-5bfa-42fa-aeaf-2cafc9654b75,Mouse,2025-04-22T02:36:09.232Z


In [0]:
# 📊 Monitoramento e Governança - Log de Execução do Pipeline Gold (Dimensão Produto)

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from datetime import datetime
import time

# ------------------------------------------
# 1. Início da contagem do tempo de execução
# ------------------------------------------
start_time = time.time()

# ------------------------------------------
# 2. Parâmetros do log
# ------------------------------------------
job_name = "gold_dim_produto"
status = "SUCESSO"
erro = None

try:
    # Contagem de registros distintos no DataFrame final da Gold
    qtd_linhas = dim_product_df.select("ItemID").distinct().count()

except Exception as e:
    status = "ERRO"
    erro = str(e)
    qtd_linhas = 0

# ------------------------------------------
# 3. Cálculo do tempo total de execução (em segundos)
# ------------------------------------------
tempo_total = round(time.time() - start_time, 2)

# ------------------------------------------
# 4. Esquema explícito do log para evitar erro de inferência
# ------------------------------------------
schema_log = StructType([
    StructField("job_name", StringType(), True),
    StructField("data_execucao", StringType(), True),
    StructField("qtd_linhas", IntegerType(), True),
    StructField("status", StringType(), True),
    StructField("erro", StringType(), True),
    StructField("tempo_total_segundos", DoubleType(), True)
])

# ------------------------------------------
# 5. Criação do DataFrame com os dados do log
# ------------------------------------------
log_execucao_df = spark.createDataFrame([(
    job_name,
    datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    qtd_linhas,
    status,
    erro,
    tempo_total
)], schema=schema_log)

# ------------------------------------------
# 6. Escrita do log no Delta Lake no container Gold
# ------------------------------------------
log_execucao_df.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .save("abfss://gold@dlsprojetofixo.dfs.core.windows.net/log_execucoes_gold_dim_produto")

print(f"📌 Log da execução do job '{job_name}' registrado com sucesso.")


📌 Log da execução do job 'gold_dim_produto' registrado com sucesso.


In [0]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.log_execucoes_gold_dim_produto
    USING DELTA
    LOCATION 'abfss://gold@dlsprojetofixo.dfs.core.windows.net/log_execucoes_gold_dim_produto'
""")

DataFrame[]

In [0]:
%sql
SELECT * FROM gold.log_execucoes_gold_dim_produto

job_name,data_execucao,qtd_linhas,status,erro,tempo_total_segundos
gold_dim_produto,2025-04-22 02:33:51,0,ERRO,name 'dim_product_df' is not defined,0.0
gold_dim_produto,2025-04-22 02:38:07,29945,SUCESSO,null,6.46
